<a href="https://colab.research.google.com/github/acrossariver/acrossariver.github.io/blob/master/%E8%87%AA%E4%BD%9C%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日経２２５スクリーニング


In [3]:
import yfinance as yf
import pandas as pd
import datetime
import requests
import time

# 検索条件
MAX_PER = 15  # 20倍
MAX_PBR = 3.0  # 2.0以下
MIN_UP_CHANGE_RATE = 0.10  # 10%以上の上昇
MIN_DOWN_CHANGE_RATE = -0.01  # 5%以上の下落
MIN_DIVIDEND_YIELD = 1.00  # 1%以上の配当利回り
DAYS = 31 #過去何日から起算しての変化率か？

import pandas as pd

def get_nikkei225_tickers():
    # CSVファイルのURL
    url = "https://raw.githubusercontent.com/nakachan-ing/finance-streamlit/master/nikkei.csv"

    # CSVファイルをpandasで読み込む (shift_jisエンコーディングを指定)
    try:
        df = pd.read_csv(url, encoding='shift_jis')
    except Exception as e:
        print(f"CSVファイルの読み込みエラー: {e}")
        return []

    # データフレームの内容を確認
    print(df.head())  # 最初の5行を表示

    # 銘柄コードを抽出し、'.T'を追加
    tickers = df['コード'].astype(str).apply(lambda x: x + '.T').tolist()

    return tickers

# 銘柄コードを取得
tickers = get_nikkei225_tickers()

print(f"取得した銘柄数: {len(tickers)}")



# 上昇・下落銘柄を格納するリスト
up_results = []
down_results = []

# スクリーニング処理
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        per = info.get("trailingPE", None)
        pbr = info.get("priceToBook", None)
        dividend_yield = info.get("dividendYield", None)

        if per is None or pbr is None or dividend_yield is None:
            continue

        # 配当利回りとPER、PBRの条件を満たしている銘柄
        if dividend_yield >= MIN_DIVIDEND_YIELD and per <= MAX_PER and pbr <= MAX_PBR:
            # 過去2週間の株価取得
            end = datetime.datetime.today()
            start = end - datetime.timedelta(days=DAYS)
            hist = stock.history(start=start, end=end)

            if len(hist) < 2:
                continue

            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            change_rate = (end_price - start_price) / start_price

            # 上昇銘柄
            if change_rate >= MIN_UP_CHANGE_RATE:
                up_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "上昇"
                })
            # 下落銘柄
            elif change_rate <= MIN_DOWN_CHANGE_RATE:
                down_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "下降"
                })

        # time.sleep(1.5)  # アクセス制限回避のためにスリープ
    except Exception as e:
        print(f"エラー発生: {ticker}, {e}")

#過去何日前から起算して
print(f"過去{DAYS}日前から起算して")

# 銘柄コードと銘柄名の対応表を作成
df_name_map = pd.read_csv("https://raw.githubusercontent.com/nakachan-ing/finance-streamlit/master/nikkei.csv", encoding='shift_jis')
df_name_map['銘柄コード'] = df_name_map['コード'].astype(str) + ".T"
df_name_map = df_name_map[['銘柄コード', '銘柄名']]

# 上昇銘柄のデータフレーム
df_up = pd.DataFrame(up_results)
df_up = pd.merge(df_up, df_name_map, on='銘柄コード', how='left')

# 表示順を調整（銘柄コードと銘柄名を先頭に）
columns = ['銘柄コード', '銘柄名'] + [col for col in df_up.columns if col not in ['銘柄コード', '銘柄名']]
df_up = df_up[columns]

print(f"\n◎{MIN_UP_CHANGE_RATE * 100:.0f}%以上上昇した銘柄数: {len(df_up)}")
df_up = df_up.sort_values(by="株価変化率", ascending=False).reset_index(drop=True)
display(df_up)

# 下落銘柄のデータフレーム
df_down = pd.DataFrame(down_results)
df_down['株価変化率_num'] = df_down['株価変化率'].str.replace('%', '').astype(float)
df_down = df_down.sort_values(by="株価変化率_num").reset_index(drop=True)
df_down = df_down.drop(columns=["株価変化率_num"])

df_down = pd.merge(df_down, df_name_map, on='銘柄コード', how='left')

columns = ['銘柄コード', '銘柄名'] + [col for col in df_down.columns if col not in ['銘柄コード', '銘柄名']]
df_down = df_down[columns]

print(f"\n◎{MIN_DOWN_CHANGE_RATE * 100:.0f}%以上下落した銘柄数: {len(df_down)}")
display(df_down)




    コード    銘柄名          社名   業種
0  4151  協和キリン    協和キリン(株)  医薬品
1  4502     武田   武田薬品工業(株)  医薬品
2  4503  アステラス  アステラス製薬(株)  医薬品
3  4506  大日本住友  大日本住友製薬(株)  医薬品
4  4507    塩野義    塩野義製薬(株)  医薬品
取得した銘柄数: 225
過去31日前から起算して

◎10%以上上昇した銘柄数: 5


,銘柄コード,PER,PBR,配当利回り,株価変化率,上昇/下降
0,1803.T,13.849787,1.229313,2.49%,15.22%,上昇
1,1801.T,13.369059,1.522885,3.82%,14.75%,上昇
2,3863.T,4.192841,0.278485,1.77%,11.90%,上昇
3,7003.T,3.429365,1.156826,1.37%,11.07%,上昇
4,1812.T,14.224991,1.346475,3.49%,10.89%,上昇



◎-1%以上下落した銘柄数: 70


,銘柄コード,PER,PBR,配当利回り,株価変化率,上昇/下降
0,6506.T,13.495008,1.772229,2.31%,-21.01%,下降
1,8308.T,11.764707,0.908438,2.12%,-15.62%,下降
2,6724.T,11.534279,0.781540,3.67%,-15.55%,下降
3,3099.T,10.416667,1.114271,2.64%,-15.05%,下降
4,5019.T,10.183017,0.636526,4.02%,-14.93%,下降
...,...,...,...,...,...,...
65,9107.T,4.277141,0.748574,5.01%,-1.38%,下降
66,5703.T,7.122520,0.406123,4.66%,-1.25%,下降
67,9101.T,4.694827,0.730256,7.40%,-1.12%,下降
68,5108.T,14.265451,1.089011,3.88%,-1.02%,下降


# プライム市場スクリーニング


In [4]:
import yfinance as yf
import pandas as pd
import datetime
import requests
import random
from io import BytesIO

# 検索条件
MAX_PER = 15  # 15倍
MAX_PBR = 2.0  # 2.0以下
MIN_UP_CHANGE_RATE = 0.10  # 10%以上の上昇
MIN_DOWN_CHANGE_RATE = -0.05  # 5%以上の下落
MIN_DIVIDEND_YIELD = 4.00  # 4%以上の配当利回り
DAYS = 31  # 過去何日から起算しての変化率か？
PRIME_RANDOM = 333 # ランダムに絞り込むプライム市場の銘柄数

# カスタムヘッダーを定義
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# === JPXのExcelファイルから東証プライム銘柄を取得 ===
xls_url = 'https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls'

# JPXのプライム市場の銘柄をExcelから取得
response = requests.get(xls_url)
xls_file = BytesIO(response.content)

# Excelファイルを読み込む（1枚目のシートを仮定）
df = pd.read_excel(xls_file, sheet_name=0)

# カラム名整形
df.columns = df.columns.str.strip()
df = df.dropna(subset=['コード'])  # コードがNaNの行を除外

# プライム市場に絞り込み
prime_df = df[df['市場・商品区分'].str.contains('プライム', na=False)].copy()



# 銘柄コードを yfinance 形式に（4桁 + ".T"）
tickers = prime_df['コード'].astype(str).str.zfill(4) + '.T'

# ランダムに333銘柄を抽出
tickers_list = random.sample(tickers.tolist(), PRIME_RANDOM)

# 取得銘柄数
print(f"取得銘柄数: {len(tickers_list)}")

# 上昇・下落銘柄を格納するリスト
up_results = []
down_results = []

# スクリーニング処理
for ticker in tickers_list:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        per = info.get("trailingPE", None)
        pbr = info.get("priceToBook", None)
        dividend_yield = info.get("dividendYield", None)

        if per is None or pbr is None or dividend_yield is None:
            continue

        # 配当利回りとPER、PBRの条件を満たしている銘柄
        if per <= MAX_PER and pbr <= MAX_PBR and dividend_yield >= MIN_DIVIDEND_YIELD:
            # 過去2週間の株価取得
            end = datetime.datetime.today()
            start = end - datetime.timedelta(days=DAYS)
            hist = stock.history(start=start, end=end)

            if len(hist) < 2:
                continue

            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            change_rate = (end_price - start_price) / start_price

            # 上昇銘柄
            if change_rate >= MIN_UP_CHANGE_RATE:
                up_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "上昇"
                })
            # 下落銘柄
            elif change_rate <= MIN_DOWN_CHANGE_RATE:
                down_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "下降"
                })

        # time.sleep(1.5)  # アクセス制限回避のためにスリープ
    except Exception as e:
        print(f"エラー発生: {ticker}, {e}")

# 過去何日前から起算して
print(f"過去{DAYS}日前から起算して")

# --- 銘柄コードと銘柄名の対応表を prime_df から取得 ---
prime_df['銘柄コード'] = prime_df['コード'].astype(str).str.zfill(4) + '.T'
df_name_map = prime_df[['銘柄コード', '銘柄名']].copy()

# 上昇銘柄のデータフレーム
df_up = pd.DataFrame(up_results)
df_up = pd.merge(df_up, df_name_map, on='銘柄コード', how='left')

# 表示順を調整（銘柄コードと銘柄名を先頭に）
columns = ['銘柄コード', '銘柄名'] + [col for col in df_up.columns if col not in ['銘柄コード', '銘柄名']]
df_up = df_up[columns]

print(f"\n◎{MIN_UP_CHANGE_RATE * 100:.0f}%以上上昇した銘柄数: {len(df_up)}")
df_up = df_up.sort_values(by="株価変化率", ascending=False).reset_index(drop=True)
display(df_up)

# 下落銘柄のデータフレーム
df_down = pd.DataFrame(down_results)
df_down['株価変化率_num'] = df_down['株価変化率'].str.replace('%', '').astype(float)
df_down = df_down.sort_values(by="株価変化率_num").reset_index(drop=True)
df_down = df_down.drop(columns=["株価変化率_num"])

df_down = pd.merge(df_down, df_name_map, on='銘柄コード', how='left')
columns = ['銘柄コード', '銘柄名'] + [col for col in df_down.columns if col not in ['銘柄コード', '銘柄名']]
df_down = df_down[columns]

print(f"\n◎{MIN_DOWN_CHANGE_RATE * 100:.0f}%以上下落した銘柄数: {len(df_down)}")
display(df_down)



取得銘柄数: 333
過去31日前から起算して

◎10%以上上昇した銘柄数: 6


<ipython-input-4-9779c012c8f8>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prime_df['銘柄コード'] = prime_df['コード'].astype(str).str.zfill(4) + '.T'


,銘柄コード,銘柄名,PER,PBR,配当利回り,株価変化率,上昇/下降
0,8043.T,スターゼン,10.917147,0.867960,2.98%,32.33%,上昇
1,1952.T,新日本空調,12.019342,1.430354,4.85%,18.16%,上昇
2,3863.T,日本製紙,4.192841,0.278485,1.77%,11.90%,上昇
3,8715.T,アニコム ホールディングス,14.904810,1.604336,2.69%,11.84%,上昇
4,1815.T,鉄建建設,13.623844,0.555921,4.40%,11.02%,上昇
5,9991.T,ジェコス,9.115538,0.580679,4.98%,10.32%,上昇



◎-5%以上下落した銘柄数: 39


,銘柄コード,銘柄名,PER,PBR,配当利回り,株価変化率,上昇/下降
0,6866.T,日置電機,12.738712,1.979719,3.47%,-18.16%,下降
1,6526.T,ソシオネクスト,12.033212,1.926589,3.37%,-16.76%,下降
2,4004.T,レゾナック・ホールディングス,6.337145,0.700748,5.04%,-13.12%,下降
3,8331.T,千葉銀行,13.071582,0.747251,3.60%,-12.54%,下降
4,8707.T,岩井コスモホールディングス,7.358385,0.731024,6.88%,-12.43%,下降
5,6807.T,日本航空電子工業,13.280252,1.151966,2.63%,-12.42%,下降
6,5020.T,ＥＮＥＯＳホールディングス,8.012725,0.596703,3.75%,-11.45%,下降
7,7240.T,ＮＯＫ,8.123641,0.535523,5.15%,-11.24%,下降
8,5821.T,平河ヒューテック,9.487951,0.423873,3.84%,-11.00%,下降
9,7976.T,三菱鉛筆,11.049777,0.982779,2.03%,-10.77%,下降
